<a href="https://colab.research.google.com/github/Nirav-Madhani/GEoBigData/blob/main/PySparks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [ ]:
!wget -q https://dlcdn.apache.org/spark/spark-3.2.0/spark-3.2.0-bin-hadoop3.2.tgz

In [ ]:
!ls

drive  sample_data  spark-3.2.0-bin-hadoop3.2.tgz


In [ ]:
!gunzip spark-3.2.0-bin-hadoop3.2.tgz
!tar xf spark-3.2.0-bin-hadoop3.2.tar

In [ ]:
!pip install -q findspark

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.2.0-bin-hadoop3.2"

In [ ]:
import findspark
findspark.init()
findspark.find()

'/content/spark-3.2.0-bin-hadoop3.2'

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder\
        .master("local")\
        .appName("Colab")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

In [ ]:
spark

In [ ]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip
get_ipython().system_raw('./ngrok http 4050 &')
!curl -s http://localhost:4040/api/tunnels

--2021-11-08 03:19:05--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 52.202.168.65, 54.237.133.81, 54.161.241.46, ...
Connecting to bin.equinox.io (bin.equinox.io)|52.202.168.65|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13832437 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  13.19M  3.34MB/s    in 4.9s    

2021-11-08 03:19:11 (2.67 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [13832437/13832437]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   
{"tunnels":[],"uri":"/api/tunnels"}


In [ ]:
get_ipython().system_raw('./ngrok http 4050 &')
!curl -s http://localhost:4040/api/tunnels

{"tunnels":[],"uri":"/api/tunnels"}


In [ ]:
import time
start_time = time.time()
df = spark.read.csv("drive/MyDrive/BigData/geonames.csv", header=True, inferSchema=True)
print(f"Time taken to load csv : {time.time()-start_time}")

Time taken to load csv : 41.61157584190369


In [ ]:
df.describe()

DataFrame[summary: string, geonameid: string, name: string, asciiname: string, alternatenames: string, latitude: string, longitude: string, feature class: string, feature code: string, country code: string, cc2: string, admin1 code: string, admin2 code: string, admin3 code: string, admin4 code: string, population: string, elevation: string, dem: string, timezone: string, modification date: string]

In [ ]:
from pyspark.sql.types import FloatType
from pyspark.sql.functions import udf, col
import pyspark.sql.functions as F
def FIND(x,y,df):
  @udf(returnType=FloatType()) 
  def dist(x1,y1):
    if x1 is None or y1 in None: return math.inf
    x1=eval(x1)
    y1=eval(y1)
    return abs(x1-x)+abs(y1-y)
  df = df.withColumn('distance',dist(df['latitude'],df['longitude']))
  df.registerTempTable('Table')
  sqlContext = SQLContext(spark)
  sqlContext.sql('select * from Table where distance = (select min(distance) from Table)').show()

In [ ]:
from pyspark.sql.functions import udf, col
from pyspark.sql import SQLContext


In [ ]:
#df = spark.read.csv("drive/MyDrive/BigData/worldcitiespop.csv", header=True, inferSchema=True)
FIND(10,25,df)

/content/spark-3.2.0-bin-hadoop3.2/python/pyspark/sql/dataframe.py:140: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  FutureWarning
/content/spark-3.2.0-bin-hadoop3.2/python/pyspark/sql/context.py:79: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning


Py4JJavaError: ignored

In [ ]:
sqlContext.sql('select * from Table where distance = (select min(distance) from Table)').show(3)

# Reverse Geocoding

## Sparks

In [ ]:
import math
from math import sqrt
def find1(x,y):
  dist = math.inf
  dataCollect = df.collect()
  ans = {}
  for row in dataCollect:
    dist1 = sqrt(   (x-row['lat'])**2  +  (y-row['lon'])**2 )
    if dist1 < dist:
      ans['name'] = row['name']
      ans['admin1'] = row['admin1']
      ans['admin2'] = row['admin2']
      ans['cc'] = row['cc']
      dist = dist1
  return ans
#'''
def find2(x,y):
  dist = math.inf
  dataCollect = df.collect()
  ans = {}
  for row in dataCollect:
    dist1 = sqrt(   (x-row['lat'])**2  +  (y-row['lng'])**2 )
    if dist1 < dist:
      ans['city'] = row['city']
      ans['population'] = row['population']
      ans['id'] = row['id']
      ans['country'] = row['country']
      dist = dist1
  return ans
#'''
#'''
def find3(x,y):
  dist = math.inf
  dataCollect = df.collect()
  ans = {}
  for row in dataCollect:
    dist1 = sqrt(   (x-row['Latitude'])**2  +  (y-row['Longitude'])**2 )
    if dist1 < dist:
      ans['City'] = row['City']
      ans['Population'] = row['Population']
      ans['Region'] = row['Region']
      ans['Country'] = row['Country']
      dist = dist1
  return ans
#'''

find = find3

start_time = time.time()
print(find(-37.81, 144.96))
print("--- %s seconds ---" % (time.time() - start_time))
start_time = time.time()
print(find(31.76, 35.21))
print("--- %s seconds ---" % (time.time() - start_time))

## Simple

In [ ]:
import pandas as pd
import math
from numpy import sqrt
#df = pd.read_csv("drive/MyDrive/BigData/geonames.csv")
def findClassic(x,y):
  dist = math.inf
  ans = {}
  for _, row in df.iterrows():
    dist1 = sqrt(   (x-row['latitude'])**2  +  (y-row['longitude'])**2 )
    if dist1 < dist:
      ans['City'] = row['name']
      ans['Population'] = row['population']
      ans['Region'] = row['admin3 code']
      ans['Country'] = row['country code']
      dist = dist1
  return ans
#df['distance']
start_time = time.time()
print(findClassic(-37.81, 144.96))
print("--- %s seconds ---" % (time.time() - start_time))
start_time = time.time()
print(findClassic(31.76, 35.21))
print("--- %s seconds ---" % (time.time() - start_time))

KeyboardInterrupt: ignored

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11061987 entries, 0 to 11061986
Data columns (total 19 columns):
 #   Column             Dtype  
---  ------             -----  
 0   geonameid          int64  
 1   name               object 
 2   asciiname          object 
 3   alternatenames     object 
 4   latitude           float64
 5   longitude          float64
 6   feature class      object 
 7   feature code       object 
 8   country code       object 
 9   cc2                object 
 10  admin1 code        object 
 11  admin2 code        object 
 12  admin3 code        object 
 13  admin4 code        object 
 14  population         int64  
 15  elevation          float64
 16  dem                int64  
 17  timezone           object 
 18  modification date  object 
dtypes: float64(3), int64(3), object(13)
memory usage: 1.6+ GB


In [ ]:
import pandas as pd
from numpy import sqrt
start_time = time.time()
df = pd.read_csv("drive/MyDrive/BigData/worldcitiespop.csv",header='infer',dtype={'Latitude':float,'Longitude':float})
print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
from numpy import sqrt
df.describe()
def findClassic(x,y):
  df['dist'] = sqrt((x - df['latitude'])**2 + (y-df['longitude'])**2)
  mi = df['dist'].idxmin()  
  return df.iloc[mi]
start_time = time.time()
print(findClassic(31.76, 35.21))
print("--- %s seconds ---" % (time.time() - start_time))

geonameid                    8183207
name                           Gonen
asciiname                      Gonen
alternatenames       Gonen;gwnn;גונן
latitude                     31.7602
longitude                    35.2068
feature class                      P
feature code                    PPLX
country code                      IL
cc2                              NaN
admin1 code                       06
admin2 code                      NaN
admin3 code                      NaN
admin4 code                      NaN
population                         0
elevation                        NaN
dem                              766
timezone              Asia/Jerusalem
modification date         2012-02-02
dist                      0.00316836
Name: 3729508, dtype: object
--- 0.40541696548461914 seconds ---


In [ ]:
'''
!pip install django
from google.colab.output import eval_js
print(eval_js("google.colab.kernel.proxyPort(8000)"))
!pip install subprocess
!python drive/MyDrive/BigData/UI/manage.py runserver
!python drive/MyDrive/BigData/build/gui.py
'''